In [21]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torchvision.datasets as datasets
import matplotlib.pyplot as plt

In [22]:
mean_grey = 0.1307
stdd_gray = 0.3081
image_dim = 784
hidden_dim = 400
latent_dim= 20
#Vae approximates a distribution to a normal, learning the mean and deviation
#transforms = transforms.Compose([transforms.ToTensor(),transforms.Normalize((mean_grey),(stdd_gray))])
transforms = transforms.Compose([transforms.ToTensor()])
                                 
train_dataset = datasets.MNIST(root='.././data',
                           train=True,
                           transform=transforms,
                           download=True)
test_dataset = datasets.MNIST(root='.././data',
                           train=False,
                           transform=transforms)

batch_size = 128
epochs = 10

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting .././data\MNIST\raw\train-images-idx3-ubyte.gz to .././data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting .././data\MNIST\raw\train-labels-idx1-ubyte.gz to .././data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting .././data\MNIST\raw\t10k-images-idx3-ubyte.gz to .././data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting .././data\MNIST\raw\t10k-labels-idx1-ubyte.gz to .././data\MNIST\raw



In [23]:
sample_dir ='results'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [30]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        #encoder
        self.fc1 = nn.Linear(image_dim, hidden_dim)
        self.fc2_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc2_stdd = nn.Linear(hidden_dim, latent_dim)
        #decoder
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, image_dim)
    
    def encode(self, x):
        h= F.relu(self.fc1(x))
        mu= self.fc2_mean(h)
        log_var = self.fc2_stdd(h)
        return mu, log_var
    
    def reparametize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)#sampling from a normal deviation
        return mu + eps * std
    def decode(self, x):
        h = F.relu(self.fc3(x))
        out = torch.sigmoid(self.fc4(h))
        return out
    def forward(self,x):
        # x: (batch_size, 1, 28,28) --> (batch_size, 784)
        mu, log_var = self.encode(x.view(-1,image_dim))
        z =  self.reparametize(mu, log_var)
        out = self.decode(z)
        return out, mu, log_var
        

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [39]:
def loss_function(reconstructed_image, original_image, mu, log_var):
    bce = F.binary_cross_entropy(reconstructed_image, original_image.view(-1,784), reduction = "sum")#reconstruction loss
    kld = 0.5*torch.sum(log_var.exp() + mu.pow(2) -1 - log_var)#sum across dimensions | kl loss
    return kld + bce

def train(epoch):
    model.train()
    train_loss = 0
    for i, (images, _) in enumerate(train_loader):
        images = images.to(device)
        reconstructed, mu, logvar = model(images)
        loss = loss_function(reconstructed, images, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()#update the weights
        
        if i % 100 == 0:
            #print(loss.tolist())
            #print(loss)
            print("Train Epoch {} [Batch {}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), loss.item()/len(images)))
            
    print('=====> Epoch {}, Average Loss: {:.3f}'.format(epoch, train_loss/len(train_loader.dataset)))
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (images, _) in enumerate(test_loader):
            images = images.to(device)
            reconstructed, mu, logvar = model(images)
            test_loss += loss_function(reconstructed, images, mu, logvar).item()
            if batch_idx == 0:
                comparison = torch.cat([images[:5], reconstructed.view(batch_size, 1, 28, 28)[:5]])
                save_image(comparison.cpu(), 'results/reconstruction_' + str(epoch) + '.png', nrow = 5)

    print('=====> Average Test Loss: {:.3f}'.format(test_loss/len(test_loader.dataset)))
    
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        # Get rid of the encoder and sample z from the gaussian ditribution and feed it to the decoder to generate samples
        sample = torch.randn(64,20).to(device)
        generated = model.decode(sample).cpu()
        save_image(generated.view(64,1,28,28), 'results/sample_' + str(epoch) + '.png')


Train Epoch 1 [Batch 0/469]	Loss: 105.592
Train Epoch 1 [Batch 100/469]	Loss: 109.340
Train Epoch 1 [Batch 200/469]	Loss: 106.749
Train Epoch 1 [Batch 300/469]	Loss: 105.978
Train Epoch 1 [Batch 400/469]	Loss: 107.097
=====> Epoch 1, Average Loss: 105.319
=====> Average Test Loss: 104.924
Train Epoch 2 [Batch 0/469]	Loss: 105.179
Train Epoch 2 [Batch 100/469]	Loss: 103.909
Train Epoch 2 [Batch 200/469]	Loss: 105.593
Train Epoch 2 [Batch 300/469]	Loss: 103.288
Train Epoch 2 [Batch 400/469]	Loss: 103.619
=====> Epoch 2, Average Loss: 105.082
=====> Average Test Loss: 104.643
Train Epoch 3 [Batch 0/469]	Loss: 103.084
Train Epoch 3 [Batch 100/469]	Loss: 104.651
Train Epoch 3 [Batch 200/469]	Loss: 102.152
Train Epoch 3 [Batch 300/469]	Loss: 106.586
Train Epoch 3 [Batch 400/469]	Loss: 103.181
=====> Epoch 3, Average Loss: 104.868
=====> Average Test Loss: 104.431
Train Epoch 4 [Batch 0/469]	Loss: 108.259
Train Epoch 4 [Batch 100/469]	Loss: 108.294
Train Epoch 4 [Batch 200/469]	Loss: 105.861
